# Query generator

This module creates a collection of queries based on filtering and join clauses collected from Join Order Benchmark. These queries are used to train, validate and test the model.

In [1]:
import json
import os

this_folder = os.path.abspath(os.getcwd())

#f = open(this_folder + "\\join_order_query_seed.json", "r")
f = open(this_folder + "\\join_order_query_seed_small.json", "r")

query_seed = json.load(f)
table_aliases = query_seed["table_aliases"]
#print(json.dumps(query_seed, indent=4))

In [2]:
def generate_queries():
    queries = set()
    for join in query_seed["joins"]:
        q_select = "SELECT "
        q_from_list_initial = set([join["table_alias1"], join["table_alias2"]])
        q_join_list = [join["join"]]

        for se in query_seed["selects"]:
            if se["table_alias"] in q_from_list_initial:
                q_select += se["select"]
                break
        else:
            continue

        for fi in query_seed["filters"]:
            q_where_initial = " WHERE "
            if fi["table_alias"] in q_from_list_initial:
                q_where_initial += fi["filter"] + " AND "
                for join2 in query_seed["joins"]:
                    q_from = " FROM "
                    q_where = q_where_initial
                    q_from_list = set([join["table_alias1"], join["table_alias2"]])
                    q_join_list = [join["join"]]

                    if join2["join"] not in q_join_list:
                        if join2["table_alias1"] in q_from_list:
                                q_join_list.append(join2["join"])
                                q_from_list.add(join2["table_alias2"])
                        elif join2["table_alias2"] in q_from_list:
                                q_join_list.append(join2["join"])
                                q_from_list.add(join2["table_alias1"]) 

                    for e in q_from_list:
                        q_from += table_aliases[e] + " AS " + e + ", "

                    for j in q_join_list:
                        q_where += j + " AND "

                    queries.add(q_select + q_from[:-2] + q_where[:-5] + ";")


    for i, q in enumerate(queries):
        if i % 3 != 0:
            with open(this_folder + "\\queries\\training_queries\\" + str(i) + ".sql", "w") as output:
                output.write(q)
        else:
            if i % 2 == 0:
                with open(this_folder + "\\queries\\validation_queries\\" + str(i) + ".sql", "w") as output:
                    output.write(q)
            else:
                with open(this_folder + "\\queries\\test_queries\\" + str(i) + ".sql", "w") as output:
                    output.write(q)

In [3]:
def generate_queries_small():
    queries = set()
    for join in query_seed["joins"]:
        q_select = "SELECT "
        q_from_list_initial = set([join["table_alias1"], join["table_alias2"]])
        q_join_list = [join["join"]]

        for se in query_seed["selects"]:
            if se["table_alias"] in q_from_list_initial:
                q_select += se["select"]
                break
        else:
            continue

        for fi in query_seed["filters"]:
            q_where_initial = " WHERE "
            if fi["table_alias"] in q_from_list_initial:
                q_where_initial += fi["filter"] + " AND "
                for join2 in query_seed["joins"]:
                    q_from = " FROM "
                    q_where = q_where_initial
                    q_from_list = set([join["table_alias1"], join["table_alias2"]])
                    q_join_list = [join["join"]] 

                    for e in q_from_list:
                        q_from += table_aliases[e] + " AS " + e + ", "

                    for j in q_join_list:
                        q_where += j + " AND "

                    queries.add(q_select + q_from[:-2] + q_where[:-5] + ";")


    for i, q in enumerate(queries):
        if i % 3 != 0:
            with open(this_folder + "\\queries\\small\\training_queries\\" + str(i) + ".sql", "w") as output:
                output.write(q)
        else:
            if i % 2 == 0:
                with open(this_folder + "\\queries\\small\\validation_queries\\" + str(i) + ".sql", "w") as output:
                    output.write(q)
            else:
                with open(this_folder + "\\queries\\small\\test_queries\\" + str(i) + ".sql", "w") as output:
                    output.write(q)

In [4]:
#generate_queries()
generate_queries_small()